In [1]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

df = pd.read_json('datasets/HotPotQA/hotpot_dev_distractor_v1.json')

## Experiment


### Graph creation and refinement section

To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [14]:
context_list = df['context']

In [24]:
context_list[6:7]

6    [[James P. Comer, [James P. Comer (born James ...
Name: context, dtype: object

In [29]:
propositions = []

for i, value in enumerate(tqdm(context_list[6:7])):
        for j, context in enumerate(value):
                index = 1
                if index < len(context):
                    words = context[1]
                    combined = " ".join(words)
                    get_propositions_nosplit(combined, propositions)

100%|██████████| 1/1 [01:37<00:00, 97.56s/it]


In [30]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

['Ed Wood is a 1994 American biographical period comedy-drama film.', 'Tim Burton directed and produced Ed Wood.', 'Johnny Depp stars as cult filmmaker Ed Wood in Ed Wood.', "The film concerns the period in Ed Wood's life when Ed Wood made his best-known films.", "The film also concerns Ed Wood's relationship with actor Bela Lugosi.", 'Martin Landau plays the role of Bela Lugosi in Ed Wood.', 'Sarah Jessica Parker is among the supporting cast of Ed Wood.', 'Patricia Arquette is among the supporting cast of Ed Wood.', 'Jeffrey Jones is among the supporting cast of Ed Wood.', 'Lisa Marie is among the supporting cast of Ed Wood.', 'Bill Murray is among the supporting cast of Ed Wood.', 'Scott Derrickson was born on July 16, 1966.', 'Scott Derrickson is an American director.', 'Scott Derrickson is a screenwriter.', 'Scott Derrickson is a producer.', 'Scott Derrickson lives in Los Angeles, California.', 'Scott Derrickson is best known for directing horror films.', "Scott Derrickson directed

In [31]:
propositions_from_file[643:]

['James P. Comer is currently the Maurice Falk Professor of Child Psychiatry at the Yale Child Study Center.',
 'James P. Comer has been the Maurice Falk Professor of Child Psychiatry at the Yale Child Study Center since 1976.',
 'James P. Comer is also an associate dean at the Yale School of Medicine.',
 "James P. Comer is one of the world's leading child psychiatrists.",
 'James P. Comer is best known for his efforts to improve the scholastic performance of children from lower-income and minority backgrounds.',
 "James P. Comer's efforts led to the founding of the Comer School Development Program in 1968.",
 'The Comer School Development Program has been used in more than 600 schools.',
 'The Comer School Development Program has been used in eighty-two school districts.',
 'James P. Comer is the author of ten books.',
 "One of James P. Comer's books is the autobiographical 'Maggie’s American Dream: The Life and Times of a Black Family', published in 1988.",
 "Another one of James P. 

In [33]:
list_of_edges = generateEdges(propositions_from_file[643:])

100%|██████████| 150/150 [07:04<00:00,  2.83s/it] 


In [34]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

Success


In [38]:
qg = QueryGraph()

if refine(qg, "person"):
    print("yes")
else:
    print("no")

yes


### Q and A section

In [22]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [ ]:
# qg = QueryGraph()
# my_dict = {}

# # Define a function to apply to each row
# def print_qa(row):
    
#     question = row['question']
#     questionid = row['_id']
#     print("question:", question)
#     result = qg.answer_question(question)

#     if result is not None:
    
#         model_answer = result['result']

#         if "don't know the answer" in model_answer:
#             model_answer = ""
    
#         if len(row['answer']) > 0:
#             real_answer = row['answer']
#         else:
#             real_answer = "\"\""

#         if len(result['intermediate_steps']) > 0:
#             cypher_query = result['intermediate_steps'][0]['query']
#             context = result['intermediate_steps'][1]['context']
       
#             if(len(context) < 1):
#             #     result = refine_query(cypher_query[6:], question)
#             #     model_answer = result['result'] 
#                 model_answer = "\"\""
#                 my_dict.update({questionid: ""})
#             else:
#                 my_dict.update({questionid: model_answer})
                
#         with open("preds.txt", "a") as preds:
#             preds.write("question: " + question + "\n")
#             preds.write("real_answer: " + real_answer + "\n")
#             preds.write("model_answer: " + model_answer + "\n")  
#             preds.write("cypher_query: " + cypher_query + "\n")
#             preds.write("======================" + "\n")  
#         preds.close()

#     else: 
#         my_dict.update({questionid: ""})
        

In [ ]:
# my_dict = {}
# # Apply the function to each row
# df[0:1].progress_apply(print_qa, axis=1)
    
# with open("dev_fullwiki_pred.json", "a") as json_file:
#     json.dump(my_dict, json_file, indent=4)

In [2]:
x = df.iloc[1]
question = x['question']
print(question)
answer = x['answer']
print(answer)

What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Chief of Protocol


In [3]:
qg = QueryGraph()
res2 = qg.answer_question(question)
res2['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (movie)-[r1]-(x)-[r2]-(y)
WHERE (
    toLower(r1.metadata) CONTAINS 'kiss and tell' OR 
    toLower(r1.metadata) CONTAINS 'corliss archer'
)
AND (
    toLower(r2.metadata) CONTAINS 'position' OR 
    toLower(r2.description) CONTAINS 'position'
)
RETURN DISTINCT y

Full Context:
[{'y': {'name': 'Chief of Protocol of the United States', 'label': 'Miscellaneous'}}, {'y': {'name': 'diplomat', 'label': 'Miscellaneous'}}]

> Finished chain.


'Chief of Protocol of the United States'

## Graph refinement 

In [6]:
qg = QueryGraph()

QUERY = """
MATCH (N) RETURN N
"""
res = qg._graph.query(QUERY)
res

[{'N': {'name': 'Ed Wood',
   'embedding': [-0.017853237688541412,
    -0.0060850814916193485,
    -0.008562407456338406,
    -0.014678412117064,
    -0.04167129844427109,
    -0.03221553936600685,
    0.009984893724322319,
    0.03831780329346657,
    -0.016836194321513176,
    -0.04197366163134575,
    0.024065449833869934,
    -0.03477189317345619,
    -0.014802106656134129,
    -0.017138557508587837,
    -0.007724033202975988,
    0.020340871065855026,
    -0.003432521829381585,
    0.006088517140597105,
    -0.02152284048497677,
    0.024175399914383888,
    0.0006820376147516072,
    0.010280385613441467,
    0.020382102578878403,
    0.022484907880425453,
    -0.008830411359667778,
    0.09543716907501221,
    -0.0139774763956666,
    0.04807592183351517,
    0.044942330569028854,
    -0.018031906336545944,
    0.023859292268753052,
    -0.05511276423931122,
    0.03391977772116661,
    0.00938703678548336,
    -0.03304016962647438,
    -0.014829594641923904,
    -0.014403535053

In [7]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [8]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

Create embeddings

In [9]:
# # Now we initialize from existing graph
# existing_graph = Neo4jVector.from_existing_graph(
#     embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
#     url=neo4j_url,
#     username=neo4j_user,
#     password=neo4j_password,
#     index_name="person_index",
#     node_label="Person",
#     text_node_properties=["name"],
#     embedding_node_property="embedding",
# )

Refactor node labels

In [10]:
# # cq = """
# # MATCH (en:Entity)
# # WHERE en.label = 'Person'
# # WITH collect(en) AS persons
# # CALL apoc.refactor.rename.label("Entity", "Person", persons)
# # YIELD batches, total, timeTaken, committedOperations
# # RETURN batches, total, timeTaken, committedOperations;
# # """
# # res = qg._graph.query(cq)

Create projection

In [11]:
cq = """
MATCH (p:Person)
RETURN gds.graph.project(
  'personproj',
  p,
  null,
  {
    sourceNodeProperties: p { .embedding },
    targetNodeProperties: {}
  }
)
"""

res = qg._graph.query(cq)

Use KNN to measure node similarities

In [12]:
cq = """
CALL gds.knn.stream('projection2', {
topK: 1,
nodeProperties: ['embedding'],
// The following parameters are set to produce a deterministic result
randomSeed: 1337,
concurrency: 1,
sampleRate: 1.0,
deltaThreshold: 0.0
})
YIELD node1, node2, similarity
WHERE similarity > .90
RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

res = qg._graph.query(cq)
res

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.knn.stream`: Caused by: java.util.NoSuchElementException: Graph with name `projection2` does not exist on database `neo4j`. It might exist on another database.}

In [ ]:
for i in res:
    for key, value in i.items():
        print(f"{key}: {value}")

Person1: Ed Wood
Person2: Ed Wood Sr
similarity: 0.9346900582313538
Person1: Ed Wood Sr
Person2: Ed Wood
similarity: 0.9346900582313538


In [ ]:
def are_dictionaries_equivalent(d1, d2):
    # Check if both have the same keys
    if d1.keys() != d2.keys():
        return False

    # Check if Person1 and Person2 are swapped but equivalent
    if (d1['Person1'] == d2['Person2'] and d1['Person2'] == d2['Person1'] and d1['similarity'] == d2['similarity']):
        return True
    
    return False


In [ ]:
newlist = res

In [ ]:
for idx, val in enumerate(res):   
    for idx2, val2 in enumerate(newlist):
        if val != val2:
            if are_dictionaries_equivalent(val, val2):
                newlist.pop(idx2)


In [ ]:
unique = []

for value in newlist:
    if value not in unique:
        unique.append(value)

unique

[{'Person1': 'Ed Wood',
  'Person2': 'Ed Wood Sr',
  'similarity': 0.9346900582313538}]

In [ ]:
node_name = "Shirley Temple"

query = f"""
MATCH (n:Person {{name: '{node_name}'}})
RETURN n
"""
print(query)

res = qg._graph.query(query)
res



MATCH (n:Person {name: 'Shirley Temple'})
RETURN n



[{'n': {'name': 'Shirley Temple',
   'embedding': [-0.0038625726010650396,
    -0.004033572040498257,
    -0.052681200206279755,
    -0.01091713272035122,
    -0.04026195406913757,
    0.00812749657779932,
    0.022437792271375656,
    0.07430087774991989,
    -0.05445154756307602,
    -0.039457254111766815,
    0.020882032811641693,
    0.001297582988627255,
    0.008328672498464584,
    -0.025549309328198433,
    -0.031302932649850845,
    0.01650981605052948,
    0.009629608131945133,
    0.05906517431139946,
    -0.006585150025784969,
    0.039189018309116364,
    0.016845108941197395,
    0.003584279678761959,
    -0.00032439574715681374,
    -0.0011844217078760266,
    0.01050807535648346,
    0.05455883964896202,
    -0.04254194721579552,
    -0.06695125997066498,
    0.024382490664720535,
    -0.030900580808520317,
    0.028835177421569824,
    -0.043400295078754425,
    -0.0028013710398226976,
    -0.008898669853806496,
    -0.004070454277098179,
    0.007135030347853899,
    

Merge similar nodes (Entity resolution)

In [ ]:
for idx, val in enumerate(unique):
    print(val)
    p1 = val['Person1']
    p2 = val['Person2']    
    cq = f"""
    MATCH (a1:Person {{name: '{p1}'}}), (a2:Person {{name: '{p2}'}})
    WITH head(collect([a1,a2])) as nodes
    CALL apoc.refactor.mergeNodes(nodes,{{
    properties:"discard",
    mergeRels:true
    }})
    YIELD node
    RETURN node;
    """
    res = qg._graph.query(cq)
    res

{'Person1': 'Ed Wood', 'Person2': 'Ed Wood Sr', 'similarity': 0.9346900582313538}
